<a href="https://colab.research.google.com/github/sarveswararao-cs/Kaggle/blob/master/aviation_fatalities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import pandas as pd
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

train_downloaded = drive.CreateFile({'id': '1SzuFoIgicCUy-wvi3X88EnwdmehLAijd'})
train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '106-BUecmSzz0-PTxk-cRu_CTlkS83cLA'})
test_downloaded.GetContentFile('test.csv') 

In [0]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('train.csv')

In [4]:
df_train.head(3)

,crew,experiment,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,...,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr,event
0,1,CA,0.011719,1,-5.28545,26.775801,-9.52731,-12.793200,16.717800,33.737499,...,37.368999,17.437599,19.201900,20.5968,-3.95115,14.5076,-4520.0,817.705994,388.829987,A
1,1,CA,0.015625,1,-2.42842,28.430901,-9.32351,-3.757230,15.969300,30.443600,...,31.170799,19.399700,19.689501,21.3547,1.33212,17.7502,-4520.0,817.705994,388.829987,A
2,1,CA,0.019531,1,10.67150,30.420200,15.35070,24.724001,16.143101,32.142799,...,-12.012600,19.396299,23.171700,22.4076,1.53786,22.2470,-4520.0,817.705994,388.829987,A


In [0]:
from sklearn.model_selection import train_test_split
labels = df_train.event.astype("category").cat.codes

In [8]:
df_train.head(2)

,crew,experiment,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,...,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr,event
0,1,CA,0.011719,1,-5.28545,26.775801,-9.52731,-12.79320,16.7178,33.737499,...,37.368999,17.437599,19.201900,20.5968,-3.95115,14.5076,-4520.0,817.705994,388.829987,A
1,1,CA,0.015625,1,-2.42842,28.430901,-9.32351,-3.75723,15.9693,30.443600,...,31.170799,19.399700,19.689501,21.3547,1.33212,17.7502,-4520.0,817.705994,388.829987,A


In [0]:
X = df_train.iloc[:, 0:27]
X_new = pd.get_dummies(X, columns = ["experiment"])

In [0]:
X_train, X_test, y_train, y_test  = train_test_split(X_new, labels, test_size = 0.4, random_state = 1)

NameError: ignored

In [11]:
X_train.shape

(2920452, 29)

In [12]:
X_test.shape

(1946969, 29)

In [13]:
y_test.head(5)
y_test.shape

(1946969,)

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
model = keras.Sequential([
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.softmax)
])

In [20]:
X_train.head(5)

,crew,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,eeg_t3,...,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr,experiment_CA,experiment_DA,experiment_SS
3922410,8,52.050781,1,7.698060,19.987900,0.580998,1.888900,-1.19586,-2.764030,20.354200,...,8.556090,12.739200,1.65224,-1.849380,2825.320068,813.830017,732.801025,1,0,0
59981,1,203.246094,0,121.087997,25.092501,101.178001,26.024099,15.59370,14.670600,8.611060,...,14.161200,28.052099,31.47230,24.467199,-18033.000000,668.056030,638.755005,1,0,0
788807,2,50.542969,0,-7.460400,-6.365550,8.006610,11.029600,-2.86997,-0.671301,-8.621780,...,-10.729200,-10.149800,-16.56020,-9.711150,-5405.810059,735.765015,1007.609985,0,1,0
2503461,5,149.174011,1,11.191000,6.582320,5.184390,3.217950,3.01418,0.532353,-0.539637,...,9.669510,15.639800,13.98130,8.046190,6425.229980,796.077026,1029.140015,0,0,1
3775714,8,117.464005,0,-25.045099,-6.315980,-4.408160,0.746784,11.04310,-2.415870,-14.786100,...,0.049671,-11.325800,-15.58050,7.285000,28576.800781,671.978027,1631.300049,1,0,0


In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=2, batch_size = 2048)

ValueError: ignored

In [0]:
l